In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn import model_selection
from sklearn import metrics
import time
import tensorflow as tf
import shap
import numpy as np

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
df = pd.read_csv('ToN-IoT_balanced_smote_lime.csv')
columns = df.columns.tolist()

In [3]:
#read kdd_processed_bin_top15.csv
df_bin = pd.read_csv('ToN-IoT_balanced_smote_top15.csv')

# get the names of the columns in a list
columns_bin = df_bin.columns.tolist()

In [4]:
#combine the two lists of columns without duplicates
columns = list(set(columns + columns_bin))
print(columns)
#print length of columns
print(len(columns))

['conn_state_OTH', 'conn_state_SHR', 'http_trans_depth_None', 'service_dns', 'dst_port', 'proto_tcp', 'label', 'proto_icmp', 'ssl_resumed_T', 'service_ssl', 'dns_AA_F', 'service_http', 'conn_state_RSTO', 'dns_AA_None', 'service_dhcp', 'http_resp_mime_types_text/plain', 'conn_state_REJ', 'src_port', 'service_None', 'service_smb', 'conn_state_RSTRH', 'service_gssapi', 'http_resp_mime_types_image/png', 'missed_bytes', 'proto_udp', 'conn_state_S3', 'conn_state_SH']
27


In [5]:
data = pd.read_csv('ToN-IoT_balanced_smote.csv')

In [6]:
data = data.drop(data.columns.difference(columns), axis=1)
data

,src_port,dst_port,missed_bytes,label,proto_icmp,proto_tcp,proto_udp,service_None,service_dhcp,service_dns,...,conn_state_RSTRH,conn_state_S3,conn_state_SH,conn_state_SHR,dns_AA_F,dns_AA_None,ssl_resumed_T,http_trans_depth_None,http_resp_mime_types_image/png,http_resp_mime_types_text/plain
0,0.101952,3.799303,0.0,0.0,False,True,False,True,False,False,...,False,False,False,False,False,True,False,True,False,False
1,2.558822,1.118792,0.0,0.0,False,False,True,True,False,False,...,False,False,False,False,False,True,False,True,False,False
2,0.101790,3.713672,0.0,0.0,False,True,False,True,False,False,...,False,False,False,False,False,True,False,True,False,False
3,1.856906,0.753177,0.0,0.0,False,True,False,True,False,False,...,False,False,False,False,False,True,False,True,False,False
4,2.523521,0.003801,0.0,0.0,False,False,True,False,False,True,...,False,False,False,True,False,True,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,3.279039,0.005737,0.0,1.0,False,True,False,True,False,False,...,False,False,True,False,False,True,False,True,False,False
2999996,1.823237,0.003801,0.0,1.0,False,False,True,False,False,True,...,False,False,False,False,True,False,False,True,False,False
2999997,2.563471,0.005737,0.0,1.0,False,True,False,True,False,False,...,False,False,True,False,False,True,False,True,False,False
2999998,1.853248,0.005737,0.0,1.0,False,True,False,True,False,False,...,False,False,True,False,False,True,False,True,False,False


In [9]:
X = data.drop(['label'], axis=1)
Y = data['label']
# Y = pd.get_dummies(Y)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)

# convert Y to int
Y_train = Y_train.astype(int)
Y_test = Y_test.astype(int)

# reshape the data for LSTM
X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

# convert all to numpy arrays
X_train = np.asarray(X_train).astype(np.float32)
# Y_train = np.asarray(Y_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)
# Y_test = np.asarray(Y_test).astype(np.float32)

In [11]:

# define the model and add recurrent dropout
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, activation = 'relu'))
model.add(Dropout(0.1))
model.add(LSTM(64, return_sequences=True, activation = 'relu'))
model.add(Dropout(0.1))
model.add(LSTM(64, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# train the model
start = time.time()
history = model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test), shuffle=True)
end = time.time()
print("Train time: ", end - start)

Epoch 1/10
75000/75000 [==============================] - 127s 2ms/step - loss: 0.0883 - accuracy: 0.9730 - val_loss: 0.0738 - val_accuracy: 0.9783
Epoch 2/10
75000/75000 [==============================] - 129s 2ms/step - loss: 0.0761 - accuracy: 0.9772 - val_loss: 0.0704 - val_accuracy: 0.9800
Epoch 3/10
75000/75000 [==============================] - 128s 2ms/step - loss: 0.0724 - accuracy: 0.9789 - val_loss: 0.0703 - val_accuracy: 0.9801
Epoch 4/10
75000/75000 [==============================] - 123s 2ms/step - loss: 0.0714 - accuracy: 0.9794 - val_loss: 0.0676 - val_accuracy: 0.9802
Epoch 5/10
75000/75000 [==============================] - 127s 2ms/step - loss: 0.0702 - accuracy: 0.9796 - val_loss: 0.0688 - val_accuracy: 0.9792
Epoch 6/10
75000/75000 [==============================] - 112s 1ms/step - loss: 0.0699 - accuracy: 0.9798 - val_loss: 0.0661 - val_accuracy: 0.9810
Epoch 7/10
75000/75000 [==============================] - 107s 1ms/step - loss: 0.0688 - accuracy: 0.9799 - val_

In [12]:
start = time.time()
y_pred = model.predict(X_test)
end = time.time()
y_pred = (y_pred > 0.5)
print('Classification Report')
print(metrics.classification_report(Y_test, y_pred, digits=4))
print('Accuracy:')
print(metrics.accuracy_score(Y_test, y_pred))
print("Predict time: ", end - start)

18750/18750 [==============================] - 9s 478us/step
Classification Report
              precision    recall  f1-score   support

           0     0.9953    0.8127    0.8948     59762
           1     0.9797    0.9996    0.9895    540238

    accuracy                         0.9810    600000
   macro avg     0.9875    0.9061    0.9422    600000
weighted avg     0.9812    0.9810    0.9801    600000

Accuracy:
0.9809633333333333
Predict time:  11.896359920501709
